<a href="https://colab.research.google.com/github/quyongkeomut/Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow---my-practice/blob/main/Chapter_14_Computer_Vision_using_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Init

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [ ]:
input_layer = keras.layers.Conv2D(25, 3, padding="same", )

##Flower Dataset  

In [ ]:
flowers_ds, flowers_ds_info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [ ]:
flowers_ds_size = flowers_ds_info.splits["train"].num_examples
flowers_ds_size

3670

In [ ]:
flowers_class_names = flowers_ds_info.features["label"].names
flowers_class_names

['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

In [ ]:
num_classes = flowers_ds_info.features["label"].num_classes
num_classes

5

However, this dataset just have one train subset "train" and do not have any other subset, such as "validation" or "test". Because of that, we split this "train" set to multiple subsets: first 10% for testing, next 15% for validation and the 75% remainer for training:

In [ ]:
flowers_test, flowers_valid, flowers_train = tfds.load("tf_flowers", split=["train[:10%]", "train[10%:25%]", "train[25%:]"], as_supervised=True)

We must also preprocess the whole subsets. This Xception requires 224 $\times$ 224 images, so we write the transform procedure:

In [ ]:
def flowers_preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

Preprocess the three subsets, shuffle them, and then pack as prefetched batches:

In [ ]:
batch_size = 32
flowers_train = flowers_train.shuffle(1000, reshuffle_each_iteration=True)

flowers_train = flowers_train.map(flowers_preprocess).batch(batch_size).prefetch(1)
flowers_valid = flowers_valid.map(flowers_preprocess).batch(batch_size).prefetch(1)
flowers_test = flowers_test.map(flowers_preprocess).batch(batch_size).prefetch(1)

## **AlexNet**

In [ ]:
class AlexNet(keras.Model):
    def __init__(self, output_dim=10, input_dim=[320, 320, 3], **kwargs):
        super().__init__(**kwargs)
        self.output_dim = output_dim
        self.input_dim = input_dim
        self.layer = [
            keras.layers.InputLayer(input_shape=self.input_dim, name="input"),
            keras.layers.Conv2D(filters=96,
                                kernel_size=11,
                                strides=4,
                                padding="valid",
                                activation="relu",
                                name="conv_layer_1"),
            keras.layers.Lambda(lambda X: tf.nn.local_response_normalization(X, depth_radius=2,
                                                                             alpha=2e-5, beta=0.75, bias=1),
                                name="local_response_normalization_layer1"),
            keras.layers.MaxPool2D(pool_size=3,
                                   strides=2,
                                   padding="valid",
                                   name="maxpool_layer_2"),
            keras.layers.Conv2D(filters=256,
                                kernel_size=5,
                                padding="same",
                                activation="relu",
                                name="conv_layer_3"),
            keras.layers.Lambda(lambda X: tf.nn.local_response_normalization(X, depth_radius=2,
                                                                             alpha=2e-5, beta=0.75, bias=1),
                                name="local_response_normalization_layer2"),
            keras.layers.MaxPool2D(pool_size=3,
                                   strides=2,
                                   padding="valid",
                                   name="maxpool_layer_4"),
            keras.layers.Conv2D(filters=384,
                                kernel_size=3,
                                padding="same",
                                activation="relu",
                                name="conv_layer_5"),
            keras.layers.Conv2D(filters=384,
                                kernel_size=3,
                                padding="same",
                                activation="relu",
                                name="conv_layer_6"),
            keras.layers.Conv2D(filters=256,
                                kernel_size=3,
                                padding="same",
                                activation="relu",
                                name="conv_layer_7"),
            keras.layers.MaxPool2D(pool_size=3,
                                   strides=2,
                                   padding="valid",
                                   name="maxpool_layer_8"),
            keras.layers.Flatten(),
            keras.layers.Dense(4096, activation="relu", name="hidden9"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(4096, activation="relu", name="hidden10"),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(self.output_dim, activation="relu", name="output")
        ]

    def build(self, batch_input_shape):
      super().build(batch_input_shape)

    def call(self, X):
        Z = X
        for layer in self.layer:
            Z = layer(Z)
        return Z

    def get_config(self):
        base_config = super().get_config()
        return {**base_config,
                "output_dim": self.output_dim,
                "input_dim": self.input_dim}

In [ ]:
alexnet_model = AlexNet(output_dim=num_classes)

In [ ]:
alexnet_model.compile(loss="sparse_categorical_crossentropy",
                      optimizer="adam",
                      metrics=["accuracy"])

In [ ]:
alexnet_model.fit(flowers_train, validation_data=flowers_valid, epochs=23)

Epoch 1/23
86/86 [==============================] - 623s 7s/step - loss: 1.6731 - accuracy: 0.2398 - val_loss: 1.6094 - val_accuracy: 0.2377
Epoch 2/23
86/86 [==============================] - 557s 6s/step - loss: 1.6094 - accuracy: 0.2402 - val_loss: 1.6094 - val_accuracy: 0.2377
Epoch 3/23
86/86 [==============================] - 567s 7s/step - loss: 1.6094 - accuracy: 0.2402 - val_loss: 1.6094 - val_accuracy: 0.2377
Epoch 4/23
86/86 [==============================] - 571s 7s/step - loss: 1.6094 - accuracy: 0.2402 - val_loss: 1.6094 - val_accuracy: 0.2377
Epoch 5/23
 7/86 [=>............................] - ETA: 7:58 - loss: 1.6094 - accuracy: 0.2500

KeyboardInterrupt: ignored

## **GoogLeNet**



### Module Inception

![picture](https://drive.google.com/uc?export=view&id=1HsOiaScv1hsuP9k8BCDjb-JCQ-ZZQjiY)

In [ ]:
class InceptionBlock(keras.layers.Layer):
    def __init__(self, num_channels_each_conv, **kwargs):
        super().__init__(**kwargs)

        self.num_channels_each_conv = num_channels_each_conv
        self.first_layers = [
            keras.layers.Conv2D(kernel_size=1,
                                filters=self.num_channels_each_conv[4],
                                padding="same",
                                activation="relu"),
            keras.layers.Conv2D(kernel_size=1,
                                filters=self.num_channels_each_conv[5],
                                padding="same",
                                activation="relu"),
            keras.layers.MaxPool2D(pool_size=3,
                                   strides=1,
                                   padding="same")
        ]
        self.second_layers = [
            keras.layers.Conv2D(kernel_size=1,
                                filters=self.num_channels_each_conv[0],
                                padding="same",
                                activation="relu"),
            keras.layers.Conv2D(kernel_size=3,
                                filters=self.num_channels_each_conv[1],
                                padding="same",
                                activation="relu"),
            keras.layers.Conv2D(kernel_size=5,
                                filters=self.num_channels_each_conv[2],
                                padding="same",
                                activation="relu"),
            keras.layers.Conv2D(kernel_size=1,
                                filters=self.num_channels_each_conv[3],
                                padding="same",
                                activation="relu"),
        ]
        self.out = keras.layers.Concatenate()

    def call(self, X):
       first_out_first_layers = self.first_layers[0](X)
       second_out_first_layers = self.first_layers[1](X)
       third_out_first_layers = self.first_layers[2](X)

       first_out_second_layers = self.second_layers[0](X)
       second_out_second_layers = self.second_layers[1](first_out_first_layers)
       third_out_second_layers = self.second_layers[2](second_out_first_layers)
       fourth_out_second_layers = self.second_layers[3](third_out_first_layers)
       return self.out([first_out_second_layers, second_out_second_layers, third_out_second_layers, fourth_out_second_layers])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "num_channels_each_conv": self.num_channels_each_conv}

### GoogeLenet architecture
![](https://drive.google.com/uc?export=view&id=10KWIclw7gzoOWLOR2VGqQoTZk8t8e94u)

In [ ]:
class GoogLeNet(keras.Model):
    def __init__(self, input_dim=[224, 224, 3], output_dim=10, **kwargs):
        super().__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layer = [
            keras.layers.InputLayer(input_shape=input_dim),
            keras.layers.Conv2D(filters=64,
                                kernel_size=7,
                                strides=2,
                                padding="same",
                                activation="relu"),
            keras.layers.MaxPool2D(pool_size=3,
                                   strides=2,
                                   padding="same"),
            keras.layers.Lambda(lambda X: tf.nn.local_response_normalization(X, depth_radius=2,
                                                                             alpha=2e-5, beta=0.75, bias=1)),
            keras.layers.Conv2D(filters=64,
                                kernel_size=1,
                                strides=1,
                                padding="same",
                                activation="relu"),
            keras.layers.Conv2D(filters=192,
                                kernel_size=3,
                                strides=1,
                                padding="same",
                                activation="relu"),
            keras.layers.Lambda(lambda X: tf.nn.local_response_normalization(X, depth_radius=2,
                                                                             alpha=2e-5, beta=0.75, bias=1)),
            keras.layers.MaxPool2D(pool_size=3,
                                   strides=2,
                                   padding="same"),
            InceptionBlock((64, 128, 32, 32, 96, 16)),
            InceptionBlock((128, 192, 96, 64, 128, 32)),
            keras.layers.MaxPool2D(pool_size=3,
                                   strides=2,
                                   padding="same"),
            InceptionBlock((192, 208, 48, 64, 96, 16)),
            InceptionBlock((160, 224, 64, 64, 112, 24)),
            InceptionBlock((128, 256, 64, 64, 128, 24)),
            InceptionBlock((112, 288, 64, 64, 144, 32)),
            InceptionBlock((256, 320, 128, 128, 160, 32)),
            keras.layers.MaxPool2D(pool_size=3,
                                   strides=2,
                                   padding="same"),
            InceptionBlock((256, 320, 128, 128, 160, 32)),
            InceptionBlock((384, 384, 128, 128, 192, 48)),
            keras.layers.GlobalAvgPool2D(),
            keras.layers.Flatten(),
            keras.layers.Dropout(0.4),
            keras.layers.Dense(output_dim, activation="softmax")
        ]

    def call(self, X):
        Z = X
        for layer in self.layer:
            Z = layer(Z)
        return Z

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "input_dim": self.input_dim,
                "output_dim": self.output_dim}

In [ ]:
googlenet = GoogLeNet()
optimizer = keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.9, nesterov=True, decay=0.01)
googlenet.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])

In [ ]:
history = googlenet.fit(flowers_train, epochs=10, validation_data=flowers_valid)

Epoch 1/10
86/86 [==============================] - 19s 188ms/step - loss: 1.7045 - accuracy: 0.2115 - val_loss: 1.5986 - val_accuracy: 0.2377
Epoch 2/10
86/86 [==============================] - 16s 183ms/step - loss: 1.3976 - accuracy: 0.3710 - val_loss: 1.2108 - val_accuracy: 0.5082
Epoch 3/10
86/86 [==============================] - 16s 183ms/step - loss: 1.1435 - accuracy: 0.5069 - val_loss: 1.0113 - val_accuracy: 0.5862
Epoch 4/10
86/86 [==============================] - 16s 183ms/step - loss: 1.0121 - accuracy: 0.5836 - val_loss: 0.9672 - val_accuracy: 0.6116
Epoch 5/10
86/86 [==============================] - 16s 183ms/step - loss: 0.9362 - accuracy: 0.6097 - val_loss: 0.9189 - val_accuracy: 0.6388
Epoch 6/10
86/86 [==============================] - 16s 184ms/step - loss: 0.8548 - accuracy: 0.6414 - val_loss: 0.8337 - val_accuracy: 0.6878
Epoch 7/10
86/86 [==============================] - 16s 182ms/step - loss: 0.8079 - accuracy: 0.6693 - val_loss: 0.8452 - val_accuracy: 0.6570

##**ResidualNet - ResNet-34**

### Residual Block of ResNet-34

In [ ]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, filter, stride=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.filter = filter
        self.stride = stride
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters=filter,
                                kernel_size=3,
                                strides=stride,
                                padding="same",
                                use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters=filter,
                                kernel_size=3,
                                strides=1,
                                padding="same",
                                use_bias=False),
            keras.layers.BatchNormalization()
        ]
        self.shortcut_layers = []
        if stride > 1:
            self.shortcut_layers = [
                keras.layers.Conv2D(filters=filter,
                                    kernel_size=1,
                                    strides=stride,
                                    padding="same",
                                    use_bias=False),
                keras.layers.BatchNormalization()
            ]

    def call(self, X):
        Z_main = X
        for layer in self.main_layers:
          Z_main = layer(Z_main)
        Z_shortcut = X
        for layer in self.shortcut_layers:
          Z_shortcut = layer(Z_shortcut)
        return self.activation(Z_main + Z_shortcut)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "filter": self.filter,
                "stride": self.stride,
                "activation": keras.activations.serialize(self.activation)}

### ResNet-34

In [ ]:
class ResNet34(keras.Model):
    def __init__(self, output_dim=10, **kwargs):
        super().__init__(**kwargs)
        self.output_dim = output_dim
        self.main_layers = [
            keras.layers.Conv2D(filters=64,
                                kernel_size=7,
                                strides=2,
                                padding="same",
                                use_bias=False),
            keras.layers.BatchNormalization(),
            keras.layers.Activation("relu"),

            keras.layers.MaxPool2D(pool_size=3,
                                   strides=2,
                                   padding="same")
        ]
        prev_filter = 64
        for res_layer_filter in [64]*3 + [128]*4 + [256]*6 + [512]*3:
            stride = 1 if res_layer_filter == prev_filter else 2
            self.main_layers += [ResidualBlock(res_layer_filter, stride)]
            prev_filter = res_layer_filter
        self.main_layers += [
                keras.layers.GlobalAvgPool2D(),
                keras.layers.Flatten(),
                keras.layers.Dense(10, activation="softmax")
            ]

    def call(self, X):
        Z = X
        for layer in self.main_layers:
            Z = layer(Z)
        return Z

    def get_config(self):
        base_config = super().get_config()
        return {**base_config,
                "output_dim": self.output_dim}

In [ ]:
resnet = ResNet34(output_dim=num_classes)
optimizer = keras.optimizers.legacy.SGD(learning_rate=0.05, momentum=0.9, nesterov=True, decay=0.01)
resnet.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
               metrics=["accuracy"])

In [ ]:
history = resnet.fit(flowers_train, epochs=17, validation_data=flowers_valid)

Epoch 1/17
86/86 [==============================] - 1992s 23s/step - loss: 1.6438 - accuracy: 0.4262 - val_loss: 187.4589 - val_accuracy: 0.1833
Epoch 2/17
37/86 [===========>..................] - ETA: 16:58 - loss: 1.2151 - accuracy: 0.5279

KeyboardInterrupt: ignored

## Xception
![picture](https://drive.google.com/uc?export=view&id=1_7QpRPmVSdmNe5JBkbcq0lDFsPTP0al9)

## SENet

### SE Block

In [ ]:
class SEBlock(keras.layers.Layer):


### Insert SE Block into convolutional units (Residual, Inception, Xception, ...)

## FCN

## **Transfer Learning by using pre-trained Model**

In this section, we will use pre-trained Xception model to classify _tf_flowers_ dataset. We make use of pre-trained low layers of Xception. First, we must import the dataset by using TensorFLow Datasets:

In [ ]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)

83683744/83683744 [==============================] - 3s 0us/step


In [ ]:
avg = keras.layers.GlobalAvgPool2D()(base_model.output)
output = keras.layers.Dense(num_classes, activation="softmax")(avg)
model = keras.Model(inputs=[base_model.input], outputs=[output])

We should freeze the shallow layers of pre-trained model while training new model:

In [ ]:
for pre_trained_layers in base_model.layers:
    pre_trained_layers.trainable = False

In [ ]:
optimizer = keras.optimizers.legacy.SGD(lr=0.2, momentum=0.9, nesterov=True, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit(flowers_train, epochs=5, validation_data=flowers_valid)

Epoch 1/5
86/86 [==============================] - 34s 243ms/step - loss: 1.5902 - accuracy: 0.7980 - val_loss: 1.1824 - val_accuracy: 0.8439
Epoch 2/5
86/86 [==============================] - 18s 212ms/step - loss: 0.4063 - accuracy: 0.9179 - val_loss: 0.7446 - val_accuracy: 0.8639
Epoch 3/5
86/86 [==============================] - 19s 216ms/step - loss: 0.1727 - accuracy: 0.9531 - val_loss: 0.6856 - val_accuracy: 0.8675
Epoch 4/5
86/86 [==============================] - 19s 218ms/step - loss: 0.1063 - accuracy: 0.9658 - val_loss: 0.6715 - val_accuracy: 0.8693
Epoch 5/5
86/86 [==============================] - 19s 216ms/step - loss: 0.0715 - accuracy: 0.9789 - val_loss: 0.6497 - val_accuracy: 0.8693


In [ ]:
for pre_trained_layers in base_model.layers:
    pre_trained_layers.trainable = True

In [ ]:
optimizer = keras.optimizers.legacy.SGD(lr=0.2, momentum=0.9, nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit(flowers_train, epochs=20, validation_data=flowers_valid)

Epoch 1/20
86/86 [==============================] - 64s 632ms/step - loss: 1.3757 - accuracy: 0.4055 - val_loss: 19.7443 - val_accuracy: 0.1996
Epoch 2/20
86/86 [==============================] - 49s 566ms/step - loss: 1.0925 - accuracy: 0.5523 - val_loss: 4.1676 - val_accuracy: 0.4156
Epoch 3/20
86/86 [==============================] - 53s 620ms/step - loss: 0.8380 - accuracy: 0.6675 - val_loss: 4.2877 - val_accuracy: 0.4247
Epoch 4/20
86/86 [==============================] - 49s 567ms/step - loss: 0.6228 - accuracy: 0.7758 - val_loss: 0.7717 - val_accuracy: 0.7350
Epoch 5/20
86/86 [==============================] - 54s 623ms/step - loss: 0.4908 - accuracy: 0.8249 - val_loss: 0.9275 - val_accuracy: 0.7350
Epoch 6/20
86/86 [==============================] - 51s 588ms/step - loss: 0.3679 - accuracy: 0.8750 - val_loss: 0.5659 - val_accuracy: 0.8185
Epoch 7/20
86/86 [==============================] - 49s 575ms/step - loss: 0.2696 - accuracy: 0.9081 - val_loss: 0.8135 - val_accuracy: 0.778